In [ ]:
from torch.nn.utils.rnn import pad_sequence
from tfrecord_lite import tf_record_iterator
from tfrecord.torch.dataset import TFRecordDataset
import torch.utils.data as tdata
import torch

import json

In [ ]:
block_config_path = '/scratch/gilbreth/akamsali/Research/Makin/ecog2txt-pytorch/conf/block_breakdowns.json'
tfrecord_path = '/depot/jgmakin/data/ecog2txt/word_sequence/tf_records'
data_path = '/scratch/gilbreth/akamsali/Research/Makin/data/ecog2txt_ak'
subjects = ['400', '401', '402', '403']


with open(block_config_path) as bf:
    block_config_all = json.load(bf)

In [ ]:
block_config = block_config_all['400']
subject_ID = '400'
filtered_files = list(
            map(lambda y: (tfrecord_path + "/EFC"+ subject_ID + "_B" + y[0] + ".tfrecord", 
            data_path + "/EFC"+ subject_ID + "_B" + y[0] + ".json"), 
                 block_config.items()))

In [ ]:
def convert_to_str(record):
    record['text_sequence'] = list(
        map(lambda y: y.decode(), record['text_sequence']))
    record['text_sequence'].append('<EOS>') 
    record['phoneme_sequence'] = list(
        map(lambda y: y.decode(), record['phoneme_sequence']))
    record['ecog_sequence'] = record['ecog_sequence'].tolist() 
    # print(len(record['phoneme_sequence']))
    return record

datasets = []
for f in filtered_files:
    # print(f)
    dataset_iterator = tf_record_iterator(f[0])
    data_in_f = [convert_to_str(d) for d in dataset_iterator]
    datasets.extend(data_in_f)

    # with open(f[1], 'w') as jf:
    #     json.dump(data_in_f, jf)

In [ ]:
import numpy as np
words = []
for dataset in datasets:
    words = words + dataset['text_sequence']


In [ ]:
phonemes = np.array(words)

In [ ]:
all_phonemes = np.unique(phonemes)

In [ ]:
all_phonemes = all_phonemes.tolist()

In [ ]:
with open('/scratch/gilbreth/akamsali/Research/Makin/ecog2txt-pytorch/conf/vocab.words.1824', 'w') as f:
    for phoneme in all_phonemes:
        f.write("%s\n" % phoneme)

In [ ]:

files = {}
for subject_id in subjects:
    block_config = block_config_all[subject_id]

    keys = list(block_config.keys())
    files[subject_id] = list(map(lambda y: tfrecord_path + "/EFC" + subject_id + "_B" + y + ".tfrecord", keys))